In [1]:
import numpy as np
import ogsim
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random 
import math
import csv

In [2]:
def media(arquivo_csv):
    propriedades = pd.read_csv(arquivo_csv,header=None,delimiter='\t')
    resultado = []
    for i in propriedades[0]:
        linha = i.split()
        if float(linha[1]) != 352 and float(linha[1]) != 1 and float(linha[4]) > 0:
            resultado.append(float(linha[4]))
    return sum(resultado)/len(resultado)

def blocos_alterados(arquivo_csv):
    propriedades = pd.read_csv(arquivo_csv,header=None,delimiter='\t')
    resultado = []
    for i in propriedades[0]:
        linha = i.split()
        if float(linha[1]) != 352 and float(linha[1]) != 1 and float(linha[4]) > 0:
            if float(linha[4]) == 1:
                resultado.append(float(linha[4]))
    return len(resultado)

def blocos_n_alterados(arquivo_csv):
    propriedades = pd.read_csv(arquivo_csv,header=None,delimiter='\t')
    resultado = []
    for i in propriedades[0]:
        linha = i.split()
        if float(linha[1]) != 352 and float(linha[1]) != 1 and float(linha[4]) > 0:
            if float(linha[4]) != 1:
                resultado.append(float(linha[4]))
    return len(resultado)

def calcula_bloco_laminar_por(por_amostra_vug):
    x = (por_amostra_vug * 78750 - 60900 * 0.2399528956) / 17850 # 3 laminas
    y = (por_amostra_vug * 78750 -38500 * 0.2399528956) / 40250 # 7 laminas
    return x,y

def calcula_bloco_laminar_perm(por_amostra_vug):
    x = (por_amostra_vug * 78750 - 60900 * 256) / 17850 # 3 laminas
    y = (por_amostra_vug * 78750 - 38500 * 256) / 40250 # 7 laminas
    return x,y

In [3]:
def distancia_euclidiana(point1, point2):
    """
    Calculate the Euclidean distance between two points in n-dimensional space.
    
    Arguments:
    point1 (tuple): Coordinates of the first point.
    point2 (tuple): Coordinates of the second point.
    
    Returns:
    float: Euclidean distance between the two points.
    """
    if len(point1) != len(point2):
        raise ValueError("Points must have the same dimensionality")
    
    squared_distance = sum((x - y) ** 2 for x, y in zip(point1, point2))
    distance = math.sqrt(squared_distance)
    return distance



def cria_mapa_het_var_3D_cil(block_value_homo_perm, x, y, z):
    dimensao_z_por = []
    dimensao_z_perm = []
    ponto_central = (x // 2, y // 2)
    raio = x // 2
    
    for i in range(z):  # Loop sobre a dimensão Z
        plano_xy_por = []
        plano_xy_perm = []
        
        for j in range(x):  # Loop sobre X
            for k in range(y):  # Loop sobre Y
                ponto = (j, k)
                
                if distancia_euclidiana(ponto_central, ponto) > raio:
                    plano_xy_por.append(0)
                    plano_xy_perm.append(0)
                else:
                    if i == 0 or i == z - 1:
                        if (
                            (k == ponto_central[0] and j != ponto_central[1]) or  # Linha vertical do ponto central
                            (j == ponto_central[1] and k != ponto_central[0]) or  # Linha horizontal do ponto central
                            (k - ponto_central[0] == j - ponto_central[1]) or  # Linha diagonal \
                            (k - ponto_central[0] == -(j - ponto_central[1]))  # Linha diagonal /
                        ):
                            plano_xy_por.append(1)
                            plano_xy_perm.append(250000)
                        else:
                            plano_xy_por.append(0.001)
                            plano_xy_perm.append(0)
                    else:
                        perm = block_value_homo_perm
                        plano_xy_perm.append(perm)
                        por = np.log(perm / 0.1038) / 0.3255 / 100  # Correlação k, phi
                        plano_xy_por.append(por)
        
        dimensao_z_por.append(plano_xy_por)
        dimensao_z_perm.append(plano_xy_perm)
    
    return dimensao_z_por, dimensao_z_perm



def alter(df1,df2,faixa,homo_block_perm):
    '''Insere heterogeneidade em posições randômicas no mapa homogêneo
    Faixa = porcentagem de blocos a serem alterados
    Os blocos são alterados seguindo uma distribuição normal com desvio padrão de 20 e media = valor da permeabilidade na matriz'''
    blocos = 17*352*17 # blocos totais
    null_blocks = 16*4*352 # null blocks totais
    blocos_alt = 0
    while blocos_alt < (blocos-null_blocks)*faixa//1: # verifica se o número de blocos alterados é menor do que a porcentagem desejada de blocos da matriz rochosa a serem alterados
        a = random.randint(0,351)
        b = random.randint(0,17*17-1)
        if df2[a][b] == homo_block_perm:
            alt_perm = np.random.normal(homo_block_perm,30) # distribuição normal dos valores de permeabilidade
            alt_por = np.log(alt_perm/0.1038)/0.3255/100 # correlação k,phi
            df1.loc[b,a] = alt_por
            df2.loc[b,a] = alt_perm
            blocos_alt +=1
    return df1 , df2

def alter_all(df1,df2,homo_block_perm):
    '''Muda todos os blocos da matriz rochosa, seguindo a distribuição normal com desvio padrão de 20 e media = valor da permeabilidade na matriz'''
    for i in df2.columns:
        index = 0
        for j in df2[i]:
            if j == homo_block_perm:
                alt_perm = np.random.normal(homo_block_perm,30) # distribuição normal dos valores de permeabilidade
                alt_por = np.log(alt_perm/0.1038)/0.3255/100 # correlação k,phi
                df1.loc[index,i] = alt_por
                df2.loc[index,i] = alt_perm
            index += 1
    return df1 , df2


def format_cmg(df,xd,yd,zd):
    '''Transforma os valores do DataFrame no formato do include
    Ex: 5 55 7 =   256 '''
    porosity = []
    y = yd # z = z(dimensão)
    for i in df.columns:
        x = xd # x = x(dimensão)
        z = zd
        for j in df[i]:
            entry = '{0} {1} {2} =   {3:.10f}'.format(x,y,z,j)
            porosity.append(entry)
            z -= 1
            if z == 0: # y = y(dimensão) + 1
                z = zd
                x -= 1
        y -= 1
    porosity_3dmap_cmg = pd.DataFrame(porosity)
    return porosity_3dmap_cmg

In [6]:
import shutil
import os
import math

# Definir os caminhos das pastas
pasta_x = r"D:\HOMOGENEO\Difusores\Estrela\Arquivos_Base"
pasta_y = r"D:\HOMOGENEO\Difusores\Estrela\Convergencia_Malha"

# Nome base do arquivo a ser copiado
nome_arquivo_base = 'arquivo_base.dat'
nome_includeKR = 'RELPERM_PC.inc'
nome_includeINJ = 'injetores_3d_17.inc'
nome_includePROD = 'produtores_3d_17.inc'

caminho_arquivo_origem = os.path.join(pasta_x, nome_arquivo_base)
print(caminho_arquivo_origem)
caminho_include_mapa = os.path.join(pasta_x, nome_includeKR)
caminho_include_injetores = os.path.join(pasta_x, nome_includeINJ)
caminho_include_produtores = os.path.join(pasta_x, nome_includePROD)


def substituir_linha(caminho_arquivo, linha_original, linha_substituta):
    """Substitui linhas específicas em um arquivo."""
    with open(caminho_arquivo, 'r') as file:
        linhas = file.readlines()

    with open(caminho_arquivo, 'w') as file:
        for linha in linhas:
            if linha.strip() == linha_original.strip():
                linha = linha_substituta + '\n'
            file.write(linha)


def distancia_euclidiana(point1, point2):
    if len(point1) != len(point2):
        raise ValueError("Points must have the same dimensionality")
    
    squared_distance = sum((x - y) ** 2 for x, y in zip(point1, point2))
    distance = math.sqrt(squared_distance)
    return distance

def pocos_injetores(nome_arquivo, grid_y):
    """Cria um poço no centro da malha."""
    ponto_central = ((grid_y // 2)+1, (grid_y // 2)+1)
    print(f"Gerando poço injetor central no ponto: {ponto_central}")

    try:
        with open(nome_arquivo, 'w') as arquivo:
            contador = 1  # Contador de poços
            arquivo.write(
                f"WELL  'INJE {contador}'\n"
                f"INJECTOR MOBWEIGHT 'INJE {contador}'\n"
                f"INCOMP  WATER\n"
                f"OPERATE  MAX  STW  0.25 CONT\n"
                f"OPERATE  MAX  BHP  1000  CONT\n"
                f"**               rad      geofac    wfrac   skin\n"
                f"*GEOMETRY  *J    1.125    0.37       1.00    0.0  \n"
                f"PERF        WI  'INJE {contador}'\n"
                f"** UBA             wi          Status  Connection  \n"
                f"{ponto_central[0]} 1 {ponto_central[1]}           -  OPEN    FLOW-FROM  'SURFACE'\n\n"
            )
            print(f"Poço criado com sucesso no ponto {ponto_central}")
    except Exception as e:
        print(f"Erro ao escrever no arquivo: {e}")

def pocos_produtores(nome_arquivo, grid_x, grid_z):
    """Cria um poço produtor no centro da malha."""
    ponto_central = ((grid_x // 2)+1, (grid_y // 2)+1)
    print(f"Gerando poço central no ponto: {ponto_central}")

    try:
        with open(nome_arquivo, 'w') as arquivo:
            contador = 1  # Contador de poços
            arquivo.write(
               f"WELL  'PRODUTOR {contador+1}'\n"
                                f"PRODUCER 'PRODUTOR {contador+1}'\n"
                                f"OPERATE  MIN  BHP  101.325  CONT \n"
                                f"**               rad      geofac    wfrac   skin\n"
                                f"GEOMETRY  J  0.1397  0.37  1.0  0.0  \n"
                                f"PERF      GEOA  'PRODUTOR {contador+1}'\n"
                                f"** UBA              ff          Status  Connection  \n"
                                f"{ponto_central[0]} {grid_z} {ponto_central[1]}      1.0  OPEN    FLOW-TO  'SURFACE'\n"
                                f"\n"
            )
            print(f"Poço criado com sucesso no ponto {ponto_central}")
    except Exception as e:
        print(f"Erro ao escrever no arquivo: {e}")
        
grid_x, grid_z, grid_y = 7, 252, 7  # Tamanho do grid
# Loop para copiar, renomear e ajustar o arquivo em 5 iterações
for i in range(1, 9):
    subpasta_iteracao = os.path.join(pasta_y, f'G{grid_x}x{grid_y}')
    os.makedirs(subpasta_iteracao, exist_ok=True)
    print(f'Os novos grids sãp {grid_x} e {grid_y}')
    # Caminhos dos arquivos
    nome_arquivo_destino =f'G{grid_x}x{grid_y}.dat'
    caminho_arquivo_destino = os.path.join(subpasta_iteracao, nome_arquivo_destino)
    print(f' o caminho é {caminho_arquivo_destino}')
          #####################################################################################################################
#SEÇÃO DE CRIAÇÃO DOS MAPAS
    por_000, perm_000 = cria_mapa_het_var_3D_cil(192,grid_x,grid_y,grid_z)

    df_000 = pd.DataFrame(por_000)

    df_000_perm = pd.DataFrame(perm_000)

    ndf_000 = df_000.T

    ndf_000_perm = df_000_perm.T

    #ndf_het_000 , ndf_het_000_perm = alter(ndf_000,ndf_000_perm,0.00,192)

    ndf_het_000_format = format_cmg(ndf_000,grid_x,grid_z,grid_y)

    ndf_het_000_perm_format = format_cmg(ndf_000_perm,grid_x,grid_z,grid_y) 


    ndf_het_000_format.to_csv('por_3d_het_000.inc',sep=' ',header=None, index = False, quoting = csv.QUOTE_NONE, escapechar = ' ')
    ndf_het_000_perm_format.to_csv('perm_3d_het_000.inc',sep=' ',header=None, index = False, quoting = csv.QUOTE_NONE, escapechar = ' ')

   
#####################################################################################################################
  
    

#####################################################################################################################
    n_total_blocos = grid_x*grid_z*grid_y
    # Cópia dos arquivos base e includes
    shutil.copy2(caminho_arquivo_origem, caminho_arquivo_destino)
    shutil.copy2(caminho_include_mapa, os.path.join(subpasta_iteracao, nome_includeKR))

    pasta_mae = r"D:\HOMOGENEO\Difusores\Estrela"
    nome_mapa_por = 'por_3d_het_000.inc'
    nome_mapa_perm = 'perm_3d_het_000.inc'    
    caminho_arquivo_por_origem = os.path.join(pasta_mae, nome_mapa_por)
    caminho_arquivo_perm_origem = os.path.join(pasta_mae, nome_mapa_perm)
    print(f'o caminho do arquivo por é {caminho_arquivo_por_origem} o perm é {caminho_arquivo_perm_origem}')
    shutil.copy2(caminho_arquivo_por_origem, os.path.join(subpasta_iteracao, nome_mapa_por))
    shutil.copy2(caminho_arquivo_perm_origem, os.path.join(subpasta_iteracao, nome_mapa_perm))


    # Substituições de linha
    substituir_linha(caminho_arquivo_destino, '*GRID *CART 1   1   1     ** 1-d 52 blocks',
                     f'*GRID *CART {grid_x}   {grid_z}   {grid_y}    ** 3-d 1000 blocks')
    substituir_linha(caminho_arquivo_destino, '*DI *CON 0.2241176471         	** I direction block dimensions (cm), equivalent Diam = 3.81 cm  ',
                     f'*DI *CON {3.81/grid_x}         	** I direction block dimensions (cm), equivalent Diam = 3.81 cm  ')
    substituir_linha(caminho_arquivo_destino, '*DJ *CON 0.0142045455		** J direction block dimensions (cm), core length = 5 cm',
                     f'*DJ *CON {5/grid_z}		** J direction block dimensions (cm), core length = 5 cm')
    substituir_linha(caminho_arquivo_destino, '*DK *CON 0.2241176471 				** K direction block dimensions (cm), equivalent Diam = 3.81 cm',
                     f'*DK *CON {3.81/grid_y} 				** K direction block dimensions (cm), equivalent Diam = 3.81 cm')
    

    substituir_linha(caminho_arquivo_destino, '*POR *JVAR  0.01  350*0.20  0.01  **ALL 2550*0.20   ** Porosity is constant throughout the reservoir. ',
                     f'*POR *JVAR  0.01  {grid_z-2}*0.20  0.01  **ALL {n_total_blocos}*0.20   ** Porosity is constant throughout the reservoir. ')
    substituir_linha(caminho_arquivo_destino, '*PERMI *JVAR 250000  350*50.0  250000      ** Each of the I, J, and K permeabilities    ',
                     f'*PERMI *JVAR 250000  {grid_z-2}*50.0  250000      ** Each of the I, J, and K permeabilities    ')
    substituir_linha(caminho_arquivo_destino, '*FLUX_ISECTOR *IJK    1:17  1  1:17     1',
                     f'*FLUX_ISECTOR *IJK    1:{grid_x}  1  1:{grid_y}     1')
    substituir_linha(caminho_arquivo_destino, '1:17  2:351  1:17 2',
                     f'1:{grid_x}  2:{grid_z-1}  1:{grid_y} 2')
    
    substituir_linha(caminho_arquivo_destino, '                      1:17  352  1:17   3 ',
                     f'1:{grid_x}  {grid_z}  1:{grid_y}   3')
    
    substituir_linha(caminho_arquivo_destino, '   *RTYPE *JVAR 2 350*1 2          ',
                     f'  *RTYPE *JVAR 2 {grid_z-2}*1 2 ')
    
    #  *SO *JVAR
    substituir_linha(caminho_arquivo_destino, '1  350*0.80  1   ',
                     f'1  {grid_z-2}*0.80  1   ')
    
    #  *PRES *JVAR 
    substituir_linha(caminho_arquivo_destino, '108 350*102 101.325',
                     f'108 {grid_z-2}*102 101.325')
    

        #INSERINDO O INCLUDE DOS POÇOS - TO COM PROBLEMA AQUI!
    injetores_nome = 'injetores_3d_17.inc'


    nome_novo_inj = f'injetores_malha{grid_x}x{grid_y}.inc'
    substituir_linha(caminho_arquivo_destino, f"*INCLUDE 'injetores_3d_17.inc'",
                     f"*INCLUDE {nome_novo_inj}")
    




    produtores_nome = 'produtores_3d_17.inc'
    nome_novo_prod = f'produtores_malha{grid_x}x{grid_y}.inc'
    substituir_linha(caminho_arquivo_destino, f"*INCLUDE 'produtores_3d_17.inc'",
                     f"*INCLUDE {nome_novo_prod}")




    # Geração dinâmica de poços
    caminho_pocos_injetores = os.path.join(subpasta_iteracao, f'injetores_malha{grid_x}x{grid_y}.inc')
    pocos_injetores(caminho_pocos_injetores, grid_y)
    
    caminho_pocos_produtores = os.path.join(subpasta_iteracao, f'produtores_malha{grid_x}x{grid_y}.inc')
    pocos_produtores(caminho_pocos_produtores, grid_y, grid_z)


    
    
    


    print(f"Iteração {i} concluída. Arquivos gerados em: {subpasta_iteracao}")
    arquivo = ogsim.IMEX(caminho_arquivo_destino)
    print(arquivo)
    cmg_sim = ogsim.IMEX.run(arquivo)
    print(nome_arquivo_destino) 


    nome_arquivo_generico = f'G{grid_x}x{grid_y}'
    Caminho_log = rf"C:\Users\dougalmeida\Desktop\LRAP\WORKPLACE\dg\{nome_arquivo_generico}.log"
    subpasta_iteracao = rf"C:\Users\dougalmeida\Desktop\LRAP\WORKPLACE\dg\iteracao_{i}"
    arquivo_destino = os.path.join(subpasta_iteracao, f'{nome_arquivo_generico}.log')
    
    # Certifique-se de que a subpasta existe
    os.makedirs(subpasta_iteracao, exist_ok=True)
    
#     Verificar se o arquivo de origem existe
    if os.path.exists(Caminho_log):
            #Copiar o arquivo de origem para o destino
        shutil.copy2(Caminho_log, arquivo_destino)
        print(f"Arquivo {Caminho_log} copiado para {arquivo_destino} com sucesso!")
    else:
        print(f"Erro: Arquivo {Caminho_log} não encontrado.")
            

        grid_x = grid_x + 2
        grid_y = grid_y + 2
        grid_z = grid_z + 20


print("Processo concluído com sucesso!")


D:\HOMOGENEO\Difusores\Estrela\Arquivos_Base\arquivo_base.dat
Os novos grids sãp 7 e 7
 o caminho é D:\HOMOGENEO\Difusores\Estrela\Convergencia_Malha\G7x7\G7x7.dat
o caminho do arquivo por é D:\HOMOGENEO\Difusores\Estrela\por_3d_het_000.inc o perm é D:\HOMOGENEO\Difusores\Estrela\perm_3d_het_000.inc
Gerando poço injetor central no ponto: (4, 4)
Poço criado com sucesso no ponto (4, 4)
Gerando poço central no ponto: (4, 4)
Poço criado com sucesso no ponto (4, 4)
Iteração 1 concluída. Arquivos gerados em: D:\HOMOGENEO\Difusores\Estrela\Convergencia_Malha\G7x7
IMEX Simulation
Base File Name: D:\HOMOGENEO\Difusores\Estrela\Convergencia_Malha\G7x7\G7x7.dat
G7x7.dat
Erro: Arquivo C:\Users\dougalmeida\Desktop\LRAP\WORKPLACE\dg\G7x7.log não encontrado.
Os novos grids sãp 9 e 9
 o caminho é D:\HOMOGENEO\Difusores\Estrela\Convergencia_Malha\G9x9\G9x9.dat
o caminho do arquivo por é D:\HOMOGENEO\Difusores\Estrela\por_3d_het_000.inc o perm é D:\HOMOGENEO\Difusores\Estrela\perm_3d_het_000.inc
Gerando